In [201]:
import csv
import re
import math

import plotly.graph_objects as go

from collections import Counter

In [202]:
def find_query(events):
    """
    Поиск слов не являющихся префиксом для любого другого слова в списке
    """
    queries = []
    for e in events:
        if e not in ''.join(events - {e}):
            queries.append(e.capitalize())
    return queries

In [203]:
CATEGORIES = {'Продукты', 'Где поесть', 'Гостиница', 'Достопримечательность', 'Wifi',
              'Транспорт', 'Заправка', 'Парковка', 'Шоппинг', 'Банкомат', 'Ночная жизнь',
              'Отдых с детьми', 'Банк', 'Развлечения', 'Больница', 'Аптека', 'Полиция',
              'Туалет', 'Почта'}
CATEGORIES_EN = {'Where to eat', 'Hotel', 'Groceries', 'Sights', 'Wifi', 'Transport', 'Gas',
                 'Parking', 'Shopping', 'Atm', 'Nightlife', 'Family holiday', 'Bank',
                 'Entertainment', 'Hospital', 'Pharmacy', 'Police', 'Toilet', 'Post'}

In [204]:
moscow_x = [37.547149658203125, 37.386474609375, 37.316436767578125, 37.45513916015625,
            37.651519775390625, 37.88360595703125, 37.85614013671875, 37.547149658203125]
moscow_y = [55.93304863776238, 55.88301417578358, 55.76034990679016, 55.598523085310276,
            55.5519418606257, 55.65279803318956, 55.85989956952263, 55.93304863776238]


def in_moscow(x, y):
    """
    Задача о принадлежности точки многоугольнику (МКАД)
    """
    c = 0
    for i in range(len(moscow_x)):
        if (moscow_y[i] <= y < moscow_y[i - 1] or moscow_y[i - 1] <= y < moscow_y[i]) and \
                (x > (moscow_x[i - 1] - moscow_x[i]) *
                 (y - moscow_y[i]) / (moscow_y[i - 1] - moscow_y[i]) + moscow_x[i]):
            c = 1 - c
    if c:
        return True
    return False

In [205]:
top = Counter()
top_moscow = Counter()
top_en = Counter()
top_en_moscow = Counter()
top_categories = Counter()
top_categories_en = Counter()


def process_current_user_events(events, locale, is_moscow):
    """
    Обработка запросов пользователя и распределение по топам
    """
    query = find_query(events)
    for q in query:
        if locale.startswith('ru'):
            if q in CATEGORIES:
                top_categories[q] += 1
            elif q in CATEGORIES_EN:
                top_categories_en[q] += 1
            elif is_moscow:
                top[q] += 1
                top_moscow[q] += 1
            else:
                top[q] += 1
        elif locale.startswith('en'):
            if q in CATEGORIES_EN:
                top_categories_en[q] += 1
            elif q in CATEGORIES:
                top_categories[q] += 1
            elif is_moscow:
                top_en[q] += 1
                top_en_moscow[q] += 1
            else:
                top_en[q] += 1
    events.clear()

In [206]:
gps = re.compile(r'\[.*\]')


def init_next_user(row):
    """
    Инициализация следующего пользователя
    """
    current_uid = row['uid']
    locale = row['locale'] if not row['locale'] is None else ''
    cur_gps = row['gps']
    is_moscow = False
    if not (cur_gps is None) and gps.match(cur_gps):
        x, y = list(map(float, cur_gps[1:-1].split(', ')))
        y = math.degrees(2 * math.atan(math.tanh(0.5 * math.radians(y))))
        is_moscow = in_moscow(x, y)
    return current_uid, locale, is_moscow

In [207]:
with open('raw_search_data.csv', 'r') as csv_file:
    search_reader = csv.DictReader(csv_file, delimiter=';')
    user_events = set()
    current_uid = None
    is_moscow = False
    locale = ''
    for row in search_reader:
        #  Пользовательские события идут по порядку, обработка происходит блоками с одинаковым uid
        if row['uid'] != current_uid:
            #  обработка запросов предыдущего пользователя
            process_current_user_events(user_events, locale, is_moscow)
            #  переход к новому пользователю
            current_uid, locale, is_moscow = init_next_user(row)
        user_events.add(row['query'])
    process_current_user_events(user_events, locale, is_moscow)

In [208]:
total = sum(top.values())
top = dict(top.most_common(30))
fig = go.Figure(
    [go.Bar(x=list(top.keys()), y=list(top.values()), name='top')],
    layout_title_text='Топ запросов для устройств с русской локализацией. Всего запросов: {}'.format(total)
)
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.show()

In [209]:
total = sum(top_en.values())
top_en = dict(top_en.most_common(30))
fig = go.Figure(
    [go.Bar(x=list(top_en.keys()), y=list(top_en.values()))],
    layout_title_text='Топ запросов для устройств с английской локализацией. Всего запросов: {}'.format(total)
)
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.show()

In [210]:
total = sum(top_moscow.values())
top_moscow = dict(top_moscow.most_common(30))
fig = go.Figure(
    [go.Bar(x=list(top_moscow.keys()), y=list(top_moscow.values()))],
    layout_title_text='Топ запросов для устройств с русской локализацией в Москве. Всего запросов: {}'.format(total)
)
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.show()

In [211]:
total = sum(top_en_moscow.values())
top_en_moscow = dict(top_en_moscow.most_common(30))
fig = go.Figure(
    [go.Bar(x=list(top_en_moscow.keys()), y=list(top_en_moscow.values()))],
    layout_title_text='Топ запросов для устройств с английской локализацией в Москве. Всего запросов: {}'.format(total)
)
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.show()

In [212]:
total = sum(top_categories.values())
top_categories = dict(top_categories.most_common(30))
fig = go.Figure(
    [go.Bar(x=list(top_categories.keys()), y=list(top_categories.values()))],
    layout_title_text='Топ запросов по категориям на русском. Всего запросов: {}'.format(total)
)
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.show()

In [213]:
total = sum(top_categories_en.values())
top_categories_en = dict(top_categories_en.most_common(30))
fig = go.Figure(
    [go.Bar(x=list(top_categories_en.keys()), y=list(top_categories_en.values()))],
    layout_title_text='Топ запросов по категориям на английском. Всего запросов: {}'.format(total)
)
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.show()

In [214]:
from IPython.display import IFrame

# Карта пользователей maps.me
IFrame(src='./kepler.gl.html', width=900, height=600)